In [15]:
import numpy as np
import matplotlib.pyplot as plt
from gurobipy import Model, GRB, quicksum

In [16]:
def sample_func(distribution_name, **dist_params):
     if hasattr(np.random, distribution_name):
        dist_func = getattr(np.random, distribution_name)
        samples = dist_func(**dist_params)
        return samples + 1 if distribution_name == "pareto" else samples
     else:
        raise ValueError(f"Unsupported distribution: {distribution_name}")

In [17]:
def majority_vote(sample_S, sample_D, B, k, eval_func, *eval_args):
    x_count = {}
    for _ in range(B):
        # choose k samples from total n samples
        sample_Sk = sample_S[:,:,np.random.choice(sample_S.shape[2], k, replace=False)]
        sample_Dk = sample_D[:,:,np.random.choice(sample_D.shape[2], k, replace=False)]
        x_k = tuple(eval_func(sample_Sk, sample_Dk, *eval_args))
        x_count[x_k] = x_count.get(x_k, 0) + 1
    
    x_max = max(x_count, key=x_count.get)
    return x_max

In [18]:
def gurobi_first_stage(sample_S, sample_D, C, Q_sp, Q_pc, R, M, H):
    # input:
    # C (unit cost for building a facility): p*1 numpy array
    # Q_sp (unit flow cost): s*p*g numpy array
    # Q_pc (unit flow cost): p*c*g numpy array
    # sample_S (supply): s*g*k numpy array
    # sample_D (demand): c*g*k numpy array
    # R (unit processing require): p*g numpy array
    # M (processing capacity): p*1 numpy array
    # H (multiplier): c*g numpy array
    # output:
    # x (open facility): p*1 numpy array
    # y_sp (flow supplier --> facility): s*p*g*k numpy array 
    # y_pc (flow facility --> consumer): p*c*g*k numpy array
    # z (multiplier): c*g*k numpy array
    s, p, g = Q_sp.shape
    c, g, k = sample_D.shape
    model = Model("network")
    model.setParam(GRB.Param.OutputFlag, 0)
    x = model.addVars(p, vtype=GRB.BINARY, name="x")
    y_sp = model.addVars(s, p, g, k, lb=0, vtype=GRB.CONTINUOUS, name="y_sp")
    y_pc = model.addVars(p, c, g, k, lb=0, vtype=GRB.CONTINUOUS, name="y_pc")
    z = model.addVars(c, g, k, lb=0, vtype=GRB.CONTINUOUS, name="z")

    obj_expr = quicksum(C[j] * x[j] for j in range(p)) \
                       + 1/k * quicksum(Q_sp[i, j, l] * y_sp[i, j, l, a] for i in range(s) for j in range(p) for l in range(g) for a in range(k))\
                        + 1/k * quicksum(Q_pc[j, i, l] * y_pc[j, i, l, a] for j in range(p) for i in range(c) for l in range(g) for a in range(k))\
                            + 1/k * quicksum(H[i, l] * z[i, l, a] for i in range(c) for l in range(g) for a in range(k))
    
    model.setObjective(obj_expr, GRB.MINIMIZE)

    model.addConstrs((quicksum(y_sp[i, j, l, a] for i in range(s)) - quicksum(y_pc[j, i, l, a] for i in range(c)) == 0 
                      for a in range(k) for l in range(g) for j in range(p)), name="flow")
    
    model.addConstrs((quicksum(y_pc[j, i, l, a] + z[i, l, a] for j in range(p)) >= sample_D[i, l, a]
                      for a in range(k) for l in range(g) for i in range(c)), name="demand")
    
    model.addConstrs((quicksum(y_sp[i, j, l, a] for j in range(p)) <= sample_S[i, l, a]
                      for a in range(k) for l in range(g) for i in range(s)), name="supply")
    
    model.addConstrs((quicksum(R[j, l] * quicksum(y_sp[i, j, l, a] for i in range(s)) for l in range(g)) <= M[j] * x[j]
                      for a in range(k) for j in range(p)), name="capacity")


    model.optimize()

    if model.status == GRB.OPTIMAL:
        # only return the discrete variable x
        x_opt = np.array([x[i].X for i in range(p)])
        # y_sp_opt = np.array([[[[y_sp[i, j, l, a].X for a in range(k)] for l in range(g)] for j in range(p)] for i in range(s)])
        # y_pc_opt = np.array([[[[y_pc[j, i, l, a].X for a in range(k)] for l in range(g)] for j in range(p)] for i in range(c)])
        # z_opt = np.array([[[z[i, l, a].X for a in range(k)] for l in range(g)] for i in range(c)])
        return x_opt #, y_sp_opt, y_pc_opt, z_opt
    else:
        print("No optimal solution found.")
        return None
    

In [19]:
def gurobi_second_stage(sample_S, sample_D, C, Q_sp, Q_pc, R, M, H, x):
    # second stage LP problem
    s, p, g = Q_sp.shape
    c, g, k = sample_D.shape
    model = Model("second_stage")
    model.setParam(GRB.Param.OutputFlag, 0)
    y_sp = model.addVars(s, p, g, k, lb=0, vtype=GRB.CONTINUOUS, name="y_sp")
    y_pc = model.addVars(p, c, g, k, lb=0, vtype=GRB.CONTINUOUS, name="y_pc")
    z = model.addVars(c, g, k, lb=0, vtype=GRB.CONTINUOUS, name="z")

    obj_expr = 1/k * quicksum(Q_sp[i, j, l] * y_sp[i, j, l, a] for i in range(s) for j in range(p) for l in range(g) for a in range(k))\
                        + 1/k * quicksum(Q_pc[j, i, l] * y_pc[j, i, l, a] for j in range(p) for i in range(c) for l in range(g) for a in range(k))\
                            + 1/k * quicksum(H[i, l] * z[i, l, a] for i in range(c) for l in range(g) for a in range(k))
    
    model.setObjective(obj_expr, GRB.MINIMIZE)

    model.addConstrs((quicksum(y_sp[i, j, l, a] for i in range(s)) - quicksum(y_pc[j, i, l, a] for i in range(c)) == 0
                        for a in range(k) for l in range(g) for j in range(p)), name="flow")
    
    model.addConstrs((quicksum(y_pc[j, i, l, a] + z[i, l, a] for j in range(p)) >= sample_D[i, l, a]
                        for a in range(k) for l in range(g) for i in range(c)), name="demand")
    
    model.addConstrs((quicksum(y_sp[i, j, l, a] for j in range(p)) <= sample_S[i, l, a]
                        for a in range(k) for l in range(g) for i in range(s)), name="supply")
    
    model.addConstrs((quicksum(R[j, l] * quicksum(y_sp[i, j, l, a] for i in range(s)) for l in range(g)) <= M[j] * x[j]
                        for a in range(k) for j in range(p)), name="capacity")
    
    model.optimize()

    if model.status == GRB.OPTIMAL:
        return model.ObjVal + sum(C[j] * x[j] for j in range(p)) 
    else:
        print("No optimal solution found.")
        return None
    

In [20]:
def comparison(parameters,a_ls,B,number_of_iterations,ratio,sample_number):
    C, Q_sp, Q_pc, R, M, H = parameters['C'], parameters['Q_sp'], parameters['Q_pc'], parameters['R'], parameters['M'], parameters['H']
    a_S, a_D = a_ls

    SAA_list = []
    majority_list = []
    for n in sample_number:
        SAA_intermediate = []
        majority_intermediate = []
        for _ in range(number_of_iterations):
            sample_S = sample_func('pareto', size=(s,g,n), a=a_S)  
            sample_D = sample_func('pareto', size=(c,g,n), a=a_D) 

            SAA = majority_vote(sample_S,sample_D, 1, n, gurobi_first_stage, C, Q_sp, Q_pc, R, M, H)
            SAA_intermediate.append(SAA)

            majority = majority_vote(sample_S, sample_D, B, int(n*ratio), gurobi_first_stage, C, Q_sp, Q_pc, R, M, H)
            majority_intermediate.append(majority)
            
        SAA_list.append(SAA_intermediate)
        majority_list.append(majority_intermediate)
    
    return SAA_list, majority_list

def evaluation(SAA_list, majority_list, parameters, a_ls, number_of_iterations, sample_number, large_number_sample):
    C, Q_sp, Q_pc, R, M, H = parameters['C'], parameters['Q_sp'], parameters['Q_pc'], parameters['R'], parameters['M'], parameters['H']
    a_S, a_D = a_ls
    large_sample_S = sample_func('pareto', size=(s,g,large_number_sample), a=a_S)  
    large_sample_D = sample_func('pareto', size=(c,g,large_number_sample), a=a_D)

    SAA_obj_list = []
    majority_obj_list = []
    for i in range(len(sample_number)):
        SAA_obj = 0
        majority_obj = 0
        for j in range(number_of_iterations):
            SAA_obj += gurobi_second_stage(large_sample_S, large_sample_D, C, Q_sp, Q_pc, R, M, H, SAA_list[i][j])
            majority_obj += gurobi_second_stage(large_sample_S, large_sample_D, C, Q_sp, Q_pc, R, M, H, majority_list[i][j])
            
        SAA_obj = SAA_obj/number_of_iterations
        majority_obj = majority_obj/number_of_iterations

        SAA_obj_list.append(SAA_obj)
        majority_obj_list.append(majority_obj)
    
    return SAA_obj_list, majority_obj_list

def figure_plot(SAA_obj_list, majority_obj_list,sample_number):
    # plot the objective values of SAA and Bagging-SAA
    _, ax = plt.subplots()
    ax.plot(sample_number, SAA_obj_list, marker = 'o', markeredgecolor = 'none', color = 'blue',linestyle = 'solid', linewidth = 2, label = 'SAA')
    ax.plot(sample_number, majority_obj_list, marker = 's', markeredgecolor = 'none', color = 'red',linestyle = 'solid', linewidth = 2, label = 'Majority Vote')
    ax.set_xlabel('Number of samples', size = 20)
    ax.set_ylabel('Objective', size = 20)
    ax.legend(loc = 'lower right')
    plt.show()
    return

In [21]:
# Run script - parameter setup
s = 3 # number of suppliers
p = 2 # number of facilities
c = 3 # number of consumers
g = 5 # number of products

# Randomly generate input parameters based on the given dimensions
parameters = {
    'C': np.random.rand(p),           # Unit cost for building a facility
    'Q_sp': np.random.rand(s, p, g),  # Unit flow cost from supplier to facility
    'Q_pc': np.random.rand(p, c, g),  # Unit flow cost from facility to customer
    'R': np.random.rand(p, g),        # Unit processing requirement
    'M': np.random.rand(p),           # Processing capacity
    'H': np.random.rand(c, g)        # Multiplier
}

# Pareto distribution parameter for S and D, make sure the supply is larger than demand
a_ls = np.array([np.random.uniform(1.9,2), np.random.uniform(2,2.1)]) 

B = 300
number_of_iterations = 5
ratio = 0.2
sample_number = np.array([2**i for i in range(3, 16)])
large_number_sample = 200000

In [22]:
SAA_list, majority_list = comparison(parameters,a_ls,B,number_of_iterations,ratio,sample_number)
SAA_obj_list, majority_obj_list = evaluation(SAA_list, majority_list, parameters, a_ls, number_of_iterations, sample_number, large_number_sample)
figure_plot(SAA_obj_list, majority_obj_list,sample_number)

KeyboardInterrupt: 